In [6]:
import numpy as np
from tqdm.auto import tqdm
import scipy.sparse as sps
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from numba import njit, prange

In [8]:
from recommenders.mfi.utils import build_conf_mat, build_pref_mat

In [9]:
train = sps.load_npz('./data/train1.npz')
test = sps.load_npz('./data/test1.npz')

In [10]:
n_users, n_items = train.shape
Cui_sparse = build_conf_mat(train)
Pui_sparse = build_pref_mat(train)

In [11]:
print(n_users, n_items)

148438 89276


In [12]:
K = 10

In [13]:
user_factors = np.random.normal(scale=1./K, size=(n_users, K))
item_factors = np.random.normal(scale=1./K, size=(n_items, K))

In [14]:
# can be used as assertion
@njit
def is_sorted(a):
    for i in range(a.size-1):
        if a[i+1] < a[i] :
            return False
    return True

In [15]:
is_sorted(Cui_sparse.row)

True

In [16]:
@njit
def nonzeros(u, rows, cols, vals):
    i = np.searchsorted(rows, u, side='left')
    j = np.searchsorted(rows, u, side='right')
    return cols[i:j], vals[i:j]

In [18]:
%%timeit
nonzeros(2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data)

1.44 µs ± 4.78 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
@njit
def nonzeros_csr(indptr, indices, data, row):
    """returns the non zeroes of a row in csr_matrix"""
    start, stop = indptr[row], indptr[row + 1]
    for i in range(start, stop):
        yield indices[i], data[i]
    # return indices[start:stop], data[start:stop]

In [20]:
Cui_csr = Cui_sparse.tocsr()

In [24]:
%%timeit
for t in nonzeros_csr(Cui_csr.indptr, Cui_csr.indices, Cui_csr.data, 2):
    pass

1.82 µs ± 6.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
@njit
def build_lin_equation(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors):
    A = YtY + regularization * np.eye(n_factors)
    b = np.zeros(n_factors)
    
    nz_cols, nz_vals = nonzeros(u, Cui_rows, Cui_cols, Cui_vals)
    for _i in range(nz_cols.size):
        i = nz_cols[_i]
        confidence = nz_vals[_i]
        factor = Y[i]

        # need to make change here I think wrt. preferences
        if confidence > 0:
            b += confidence * factor
        else:
            confidence *= -1

        A += (confidence - 1) * np.outer(factor, factor)
    return A, b

In [16]:
Y = item_factors
YtY = item_factors.T.dot(item_factors)

In [19]:
%%timeit
build_lin_equation(Y, YtY, 2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, 0.01, K)

10 µs ± 25.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
def user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors):
    # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    A, b = build_lin_equation(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)
    return np.linalg.solve(A, b)

In [21]:
@njit
def njit_user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors):
    # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    A, b = build_lin_equation(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)
    return np.linalg.solve(A, b)

In [23]:
%%timeit
user_factor(Y, YtY, 2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, 0.01, 10)

17 µs ± 55.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [25]:
%%timeit
njit_user_factor(Y, YtY, 2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, 0.01, 10)

11.3 µs ± 35.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [26]:
def least_squares(Cui_rows, Cui_cols, Cui_vals, X, Y, regularization):
    
    users, n_factors = X.shape
    YtY = Y.T.dot(Y)

    for u in tqdm(range(users)):
        X[u] = user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)

In [44]:
least_squares(Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, user_factors, item_factors, 0.01)

  0%|          | 0/148438 [00:00<?, ?it/s]

In [28]:
user_factors = np.random.normal(scale=1./K, size=(n_users, K))
item_factors = np.random.normal(scale=1./K, size=(n_items, K))

In [31]:
@njit
def njit_least_squares(Cui_rows, Cui_cols, Cui_vals, X, Y, regularization):
    
    users, n_factors = X.shape
    YtY = Y.T.dot(Y)

    for u in range(users):
        X[u] = njit_user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)

In [42]:
%%time
njit_least_squares(Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, user_factors, item_factors, 0.01)

Wall time: 3.43 s


In [33]:
user_factors = np.random.normal(scale=1./K, size=(n_users, K))
item_factors = np.random.normal(scale=1./K, size=(n_items, K))

In [36]:
@njit(parallel=True)
def njitp_least_squares(Cui_rows, Cui_cols, Cui_vals, X, Y, regularization):
    
    users, n_factors = X.shape
    YtY = Y.T.dot(Y)

    for u in prange(users):
        X[u] = njit_user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)

In [41]:
%%time
njitp_least_squares(Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, user_factors, item_factors, 0.01)

Wall time: 1.35 s
